In [3]:
import torch
import torchaudio

audio, sample_rate = torchaudio.load('audio_16k/Basta_16k.wav')
audio = audio.unsqueeze(0)
audio = audio.unsqueeze(0)
audio_08 = torch.nn.functional.interpolate(audio, scale_factor=0.8, mode='bilinear')
audio_12 = torch.nn.functional.interpolate(audio, scale_factor=1.2, mode='bilinear')
torchaudio.save('outputs/interpolation_0_8.wav', audio_08.squeeze(0).squeeze(0), sample_rate)
torchaudio.save('outputs/interpolation_1_2.wav', audio_12.squeeze(0).squeeze(0), sample_rate)

In [4]:
import librosa
import torch
import torchaudio


def naive_tempo_shift(wav, factor):
    # Convert the waveform to a PyTorch tensor
    wav_tensor = torch.from_numpy(wav)

    # output_length = int(len(wav_tensor[0]) * factor)
    # Compute the magnitude spectrogram of the audio
    spec = torch.stft(wav_tensor, n_fft=2048, hop_length=512, return_complex=True)
    # Compute the stretched waveform by inverting the spectrogram
    stretched_wav = torch.istft(spec, n_fft=2048, hop_length=int(512 / factor))

    return stretched_wav


wav, sr = librosa.load('audio_16k/Basta_16k.wav', sr=16000, mono=False)
wav_08 = naive_tempo_shift(wav, 0.8)
wav_12 = naive_tempo_shift(wav, 1.2)
torchaudio.save('outputs/naive_pitch_shift_0_8.wav', wav_08, sr)
torchaudio.save('outputs/naive_pitch_shift_1_2.wav', wav_12, sr)



KeyboardInterrupt: 

In [1]:
def time_stretch(signal, factor, win_size=1024, hop=1024//4):
    # create window
    hann_window = construct_hann_window(win_size)

    # draw two complex STFTs
    new_hop = int(hop * factor)
    stft_left = get_complex_stft(signal[:-hop], win_size, new_hop, hann_window)
    stft_right = get_complex_stft(signal[hop:], win_size, new_hop, hann_window)

    # calculate accumulated phase delta and reconstruct phase from it
    phase = get_acc_phase_delta(stft_left, stft_right)

    # reconstruct component from phase
    re, im = get_re_im_from_phase(phase)
    complex_new_stft = view_as_complex(stack([re, im], dim=-1)) * abs(stft_right))
    output = istft(complex_new_stft, win_length=win_size, hop_length=hop, window=hann_window)

    return output